test cases:

1. get the expected output
2. can throw out errors (unfitted, format error, etc.)
3. produce correct (or nearly correct) amount of NAs

In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from PETsARD import (
    Loader,
    Splitter,
    Processor,
    Synthesizer,
    Evaluator,
    Executor
)

In [4]:
load = Loader(
    filepath='benchmark://adult-income'
)
load.load()

Loader - Benchmarker: file benchmark/adult-income.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.


In [5]:
sp = Splitter()
sp.split(load.data)

In [6]:
synth = Synthesizer(method='sdv-gaussiancopula')

In [7]:
synth.create(sp.data[1]['train'], metadata=load.metadata.to_sdv())

In [8]:
synth.fit_sample()

Synthesizer (SDV): Fitting GaussianCopula.
Synthesizer (SDV): Fitting GaussianCopula spent 5.0776 sec.


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:700: RuntimeWarning: invalid value encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)


Synthesizer (SDV): Sampling GaussianCopula # 39073 rows (same as raw) in 0.6498 sec.


In [9]:
synth.data_syn

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,60,Private,197426,11th,15,Divorced,Adm-clerical,Husband,White,Male,1811,2,38,Holand-Netherlands,>50K
1,42,Private,206876,Assoc-voc,8,Never-married,Sales,Not-in-family,White,Female,9553,20,15,United-States,>50K
2,28,Private,102742,Some-college,8,Married-AF-spouse,Machine-op-inspct,Wife,Amer-Indian-Eskimo,Male,12561,33,38,Holand-Netherlands,<=50K
3,27,Private,183718,Masters,10,Separated,Exec-managerial,Husband,White,Male,500,0,46,United-States,<=50K
4,26,Private,230920,Some-college,13,Married-spouse-absent,Prof-specialty,Unmarried,White,Female,82400,0,40,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39068,36,Private,405609,HS-grad,7,Never-married,Sales,Husband,White,Male,295,0,45,United-States,<=50K
39069,25,Private,219476,10th,10,Never-married,Adm-clerical,Husband,White,Male,9331,0,24,United-States,<=50K
39070,63,?,199024,HS-grad,11,Divorced,Other-service,Own-child,White,Female,97644,1,56,United-States,<=50K
39071,65,Self-emp-not-inc,57288,Some-college,9,Married-spouse-absent,Sales,Not-in-family,White,Female,12598,8,27,Holand-Netherlands,>50K


In [10]:
ml = Evaluator('mlutility-cluster')

In [11]:
ml.create({'syn': synth.data_syn, 'ori': sp.data[1]['train'], 'control': sp.data[1]['validation']})

In [12]:
ml.eval()

In [13]:
ml.get_global()

,ori_mean,ori_std,syn_mean,syn_std,pct_change
0,0.035654,0.00194,0.005004,0.019284,-1.5325
